In [1]:
import warnings

warnings.filterwarnings("ignore")

import itertools
import json
import math
import os
import re
import sys
from collections import Counter
from copy import deepcopy
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from networkx.algorithms.shortest_paths.weighted import dijkstra_path_length
from rich import print
from tqdm import tqdm

In [2]:
def plot_mtx(mtx, figsize=(10, 10)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.matshow(mtx, cmap=plt.cm.Blues)
    plt.plot()

In [3]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """Convert a list of string to a list of other types

    pattern: a list of types for one item.
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [4]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="i", per_letter=False, sep=",", start_row=None, end_row=None
)


print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: 19,8,6 | Last line: 12,15,2

------

Convert to:

First line: [19, 8, 6]

Last line: [12, 15, 2]

------

In [5]:
cube_coord = np.array(data, dtype="int8")
cube_coord = cube_coord + 1  # to avoid padding
cube_size = np.amax(cube_coord, axis=0).max() + 3
print(cube_size)
cubes = np.zeros((cube_size, cube_size, cube_size), dtype="int8")
for coord in cube_coord:
    cubes[tuple(coord)] = 1

25

In [6]:
conv_cube = np.zeros((3, 3, 3), dtype="int8")
for idx in [0, 2]:
    conv_cube[idx, 1, 1] = -1
    conv_cube[1, idx, 1] = -1
    conv_cube[1, 1, idx] = -1
conv_cube[1, 1, 1] = 6
print(conv_cube.sum())

0

In [7]:
def count_ext_surfaces(cube_li):
    count_surface = 0
    for i, j, k in cube_li:
        sur_cube = cubes[
            i - 1 : i + 2,
            j - 1 : j + 2,
            k - 1 : k + 2,
        ]

        count_surface += (sur_cube * conv_cube).sum()
    return count_surface

In [8]:
cube_coord = [list(cube) for cube in cube_coord]

In [9]:
print(f"Answer to Q1: {count_ext_surfaces(cube_coord)}")

Answer to Q1: 4300

In [10]:
G = nx.Graph()  # or Graph, MultiGraph, MultiDiGraph, etc

for i in tqdm(range(1, cube_size - 1)):
    for j in range(1, cube_size - 1):
        for q in range(1, cube_size - 1):
            source_node = [i, j, q]
            if source_node not in cube_coord:
                s_node_name = "-".join([str(s) for s in source_node])
                G.add_node(s_node_name)
                for k in [1, -1]:
                    dest_node = [i + k, j, q]
                    if dest_node not in cube_coord:
                        d_node_name = "-".join([str(s) for s in dest_node])
                        G.add_edge(s_node_name, d_node_name)
                    dest_node = [i, j + k, q]
                    if dest_node not in cube_coord:
                        d_node_name = "-".join([str(s) for s in dest_node])
                        G.add_edge(s_node_name, d_node_name)
                    dest_node = [i, j, q + k]
                    if dest_node not in cube_coord:
                        d_node_name = "-".join([str(s) for s in dest_node])
                        G.add_edge(s_node_name, d_node_name)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [01:57<00:00,  5.12s/it]


In [11]:
closed_cubes = []

for sub_g in nx.connected_components(G):
    if "-".join([str(s) for s in [1, 1, 1]]) not in sub_g:
        closed_cubes.extend([[int(c) for c in n.split("-")] for n in sub_g])

In [12]:
print(
    f"Answer to Q2: {count_ext_surfaces(cube_coord)+count_ext_surfaces(closed_cubes)}"
)

Answer to Q2: 2490